In [12]:
import pandas as pd
import re
import os
import numpy as np

file = '../../../data-lake/HT/BG Algae/heterocyte_vegetative_cell_ratio.csv'

# Load the mapping keys
mapping_keys_df = pd.read_csv("mapping_keys.csv")

def process_data(file):
    df = pd.read_csv(file, header=None, encoding='unicode_escape', dtype=str)
    df = df.reset_index(drop=True)

    # Extract Date and Time
    df['Date'] = df[0]
    
    # Extract Data value
    df['Data'] = df[1]
    
    # Drop the original 'Raw' column
    df = df[['Date', 'Data']]

    # Set values for Depth and QC
    df['Depth'] = 0
    df["QC"] = "N"
    
    # Convert 'Data' to numeric
    df['Data'] = pd.to_numeric(df['Data'], errors='coerce')

    # Add "00:00:00" to the Date column
    df['Date'] = df['Date'] + ' 00:00:01'
    
    df['Date'] = df['Date'].apply(pd.to_datetime, format='%d/%m/%Y %H:%M:%S', errors='coerce')
    df = df.sort_values(by='Date')
    df = df.dropna(subset=["Date"])
    print(df)

    df['Variable'] = 'heterocyte_vegetative_cell_ratio'
    #print(df)
    
    return df

def filter_and_save_data(df, variable_name, output_filename):
    # Filter rows where the variable is equal to the specified variable_name
    variable_data = df.loc[df['Variable'] == variable_name]
    #print(variable_data)

    # Extract columns needed for the filtered data
    filtered_data = variable_data.loc[:, ["Variable", "Date", "Depth", "Data", "QC"]]

    # Replace empty cells with NaN
    filtered_data.replace("", np.nan, inplace=True)

    # Convert value of different units
    if variable_name in mapping_keys_df['Params.Name'].values:
        conv_factor = mapping_keys_df.loc[mapping_keys_df['Params.Name'] == variable_name, 'Conv'].iloc[0]
        filtered_data['Data'] = pd.to_numeric(filtered_data['Data'], errors='coerce')  # Convert non-numeric values to NaN
        filtered_data['Data'] *= conv_factor

    # Specify the directory path
    directory = '../../../data-warehouse/csv/ht/wlwq'
    
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Write the filtered DataFrame to a CSV file in the specified directory
    filtered_data.to_csv(os.path.join(directory, output_filename), index=False)

# Specify the variables needed
first_column = mapping_keys_df.iloc[:, 0]
first_column.values.tolist()

# Process the data for the current file
df = process_data(file)

# Replace empty cells with NaN
df.replace("", np.nan, inplace=True)

variable = df['Variable'].values[0]
# Filter mapping_keys_df to find the row corresponding to the variable
key_row = mapping_keys_df[mapping_keys_df['Params.Name'] == variable]

# Extract the key value from the row
key_value = key_row['Key Value'].values[0] if not key_row.empty else None

# Construct the output filename
output_filename = f'WoodsLakeMiddle_{key_value.replace(" ", "")}_profile_Data.csv'
print(output_filename)

# Filter and save data
filter_and_save_data(df, variable, output_filename)


                  Date   Data  Depth QC
1  2023-05-24 00:00:01    0.0      0  N
2  2023-06-07 00:00:01   21.0      0  N
3  2023-06-22 00:00:01   29.0      0  N
4  2023-07-06 00:00:01   19.0      0  N
5  2023-07-19 00:00:01   19.0      0  N
6  2023-08-02 00:00:01   29.0      0  N
7  2023-08-10 00:00:01   21.0      0  N
8  2023-08-15 00:00:01   31.0      0  N
9  2023-08-23 00:00:01   21.0      0  N
10 2023-08-30 00:00:01   60.0      0  N
11 2023-09-06 00:00:01   66.0      0  N
12 2023-09-13 00:00:01  126.0      0  N
13 2023-09-20 00:00:01   99.0      0  N
14 2023-09-28 00:00:01   86.0      0  N
15 2023-10-04 00:00:01  276.0      0  N
16 2023-10-10 00:00:01  137.0      0  N
17 2023-10-17 00:00:01    NaN      0  N
18 2023-10-23 00:00:01    NaN      0  N
19 2023-11-02 00:00:01    NaN      0  N
20 2023-11-10 00:00:01    NaN      0  N
21 2023-11-17 00:00:01    NaN      0  N
22 2023-11-22 00:00:01    NaN      0  N
23 2023-11-29 00:00:01    NaN      0  N
24 2023-12-04 00:00:01  475.0      0  N
